In [1]:
#Importing the necessary libraries
import os
# comment out below line to enable tensorflow logging outputs
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import time
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
from absl import app, flags, logging
from absl.flags import FLAGS
import core.utils as utils
from core.yolov4 import filter_boxes
from tensorflow.python.saved_model import tag_constants
from core.config import cfg
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from imutils.video import FPS
# deep sort imports
from deep_sort import preprocessing, nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet
from playsound import playsound

In [2]:
def get_anchors(anchors_path):
    anchors = np.array(anchors_path)
    return anchors.reshape(3, 3, 2)

In [3]:
def load_config(FLAGS):
    STRIDES = np.array(cfg.YOLO.STRIDES)
    if FLAGS == 'yolov4':
        ANCHORS = get_anchors(cfg.YOLO.ANCHORS)
    XYSCALE = cfg.YOLO.XYSCALE
    NUM_CLASS = len(utils.read_class_names(cfg.YOLO.CLASSES))
    return STRIDES, ANCHORS, NUM_CLASS, XYSCALE

In [ ]:
max_cosine_distance = 0.4
nn_budget = None
nms_max_overlap = 1.0
with open("data/classes/cus_tracking.names", "r", encoding='utf-8') as f: # tracking only few specific objects
    tracked_classes = f.read().strip().split("\n")
    
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_new_1.avi', fourcc, 20.0, (640,480))    
    
class_names = utils.read_class_names(cfg.YOLO.CLASSES)
# initialize deep sort model
model_filename = 'model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
# calculate cosine distance metric
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
# initialize tracker
tracker = Tracker(metric)


# load configuration for object detector
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
STRIDES, ANCHORS, NUM_CLASS, XYSCALE = load_config('yolov4')
input_size = 416

video_path = './data/cut_1.mp4' #Input video file
vid = cv2.VideoCapture(video_path)

#Loading the trained weights for object detection algorithm
saved_model_loaded = tf.saved_model.load('./checkpoints/yolov4-416', tags=[tag_constants.SERVING])
infer = saved_model_loaded.signatures['serving_default']


ref_frm,ref_frm_lbl,store = [],[],[],
frame_num = 0
fps = FPS().start()
while True:
    grabbed, frame = vid.read()
    if grabbed==True:
        image = Image.fromarray(frame)
        frame_size = frame.shape[:2]
        image_data = cv2.resize(frame, (input_size, input_size))
        image_data = image_data / 255.
        image_data = image_data[np.newaxis, ...].astype(np.float32)
        start_time = time.time()
        batch_data = tf.constant(image_data)
        pred_bbox = infer(batch_data)
        for key, value in pred_bbox.items():
            boxes = value[:, :, 0:4]
            pred_conf = value[:, :, 4:]

        boxes, scores, classes, valid_detections = tf.image.combined_non_max_suppression(
            boxes=tf.reshape(boxes, (tf.shape(boxes)[0], -1, 1, 4)),
            scores=tf.reshape(
                pred_conf, (tf.shape(pred_conf)[0], -1, tf.shape(pred_conf)[-1])),
            max_output_size_per_class=50,
            max_total_size=50,
            iou_threshold=0.45,
            score_threshold=0.5)    

        # convert data to numpy arrays and slice out unused elements
        num_objects = valid_detections.numpy()[0]
        bboxes = boxes.numpy()[0]
        bboxes = bboxes[0:int(num_objects)]
        scores = scores.numpy()[0]
        scores = scores[0:int(num_objects)]
        classes = classes.numpy()[0]
        classes = classes[0:int(num_objects)]

        # format bounding boxes from normalized ymin, xmin, ymax, xmax ---> xmin, ymin, width, height
        original_h, original_w, _ = frame.shape
        bboxes = utils.format_boxes(bboxes, original_h, original_w)

        # store all predictions in one parameter for simplicity when calling functions
        pred_bbox = [bboxes, scores, classes, num_objects]
        names, deleted_indx = [], []
        for i in range(num_objects):
            class_indx = int(classes[i])
            class_name = class_names[class_indx]
            if class_name not in tracked_classes:
                deleted_indx.append(i)
            else:
                names.append(class_name)
        names = np.array(names)
        count = len(names)
#         print("Objects being tracked: {}".format(count))

        # delete detections that are not in tracked_classes
        bboxes = np.delete(bboxes, deleted_indx, axis=0)
        scores = np.delete(scores, deleted_indx, axis=0)

        # encode yolo detections and feed to tracker
        features = encoder(frame, bboxes)
        detections = [Detection(bbox, score, class_name, feature) for bbox, score, class_name, feature in zip(bboxes, scores, names, features)]


        #initialize color map
        cmap = plt.get_cmap('tab20b')
        colors = [cmap(i)[:3] for i in np.linspace(0, 1, 20)]

        # run non-maxima supression
        boxs = np.array([d.tlwh for d in detections])
        scores = np.array([d.confidence for d in detections])
        classes = np.array([d.class_name for d in detections])
        indices = preprocessing.non_max_suppression(boxs, classes, nms_max_overlap, scores)
        detections = [detections[i] for i in indices]

        # Call the tracker
        tracker.predict()
        tracker.update(detections)
        # update tracks
        cur_frm,cur_frm_lbl = [],[]
        for track in tracker.tracks:
            if not track.is_confirmed() or track.time_since_update > 1:
                continue
            bbox = track.to_tlbr()
            class_name = track.get_class()
            # draw bbox on screen
            color = colors[int(track.track_id) % len(colors)]
            color = [i * 255 for i in color]
            x1, y1, x2, y2 = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            x,y = int(np.ceil((x1+x2)/2)), int(np.ceil((y1+y2)/2)) #getting the center coordinates of detected objects

            cv2.circle(frame, (x, y), 4, color, -1)
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), color, 2)
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1]-30)), (int(bbox[0])+(len(class_name)+len(str(track.track_id)))*17, int(bbox[1])), color, -1)
            cv2.putText(frame, class_name + "-" + str(track.track_id),(int(bbox[0]), int(bbox[1]-10)),0, 0.75, (255,255,255),2)
   
            cur_frm.append((x,y))
            cur_frm_lbl.append(class_name)

        #Addressing the issue missmatch of detected objects in two frames
        if(len(ref_frm)>0):
            if len(cur_frm_lbl)<len(ref_frm_lbl):
                idx = [ref_frm_lbl.index(i) for i in cur_frm_lbl if i in ref_frm_lbl]
                c_x = [ref_frm[i] for i in idx]
                res = list(np.linalg.norm(np.array(cur_frm)-np.array(c_x),axis=1))
                for i in range(len(res)):
                    if res[i]>3:
                        cv2.putText(frame, "m:{} d:{:.2f}".format(cur_frm_lbl[i], res[i]),(5, 35), 0, 1.5, (0,255,0),2)
                bal = ["m:{} d:{:.2f}".format(cur_frm_lbl[i], res[i]) for i in range(len(res)) if res[i]>3]
                print(frame_num, bal)
                        
            else:
                idx = [cur_frm_lbl.index(i) for i in ref_frm_lbl if i in cur_frm_lbl]
                c_x = [cur_frm[i] for i in idx]
                res = list(np.linalg.norm(np.array(ref_frm)-np.array(c_x),axis=1))
                for i in range(len(res)):
                    if res[i]>3:
                        cv2.putText(frame, "m:{} d:{:.2f}".format(cur_frm_lbl[i], res[i]),(5, 35), 0, 1.5, (0,255,0),2)
                bal = ["m:{} d:{:.2f}".format(cur_frm_lbl[i], res[i]) for i in range(len(res)) if res[i]>3]
                print(frame_num, bal)
                        
        #Updating the reference frame (co-ordinates and labels)
        ref_frm = cur_frm   
        ref_frm_lbl = cur_frm_lbl
        result = np.asarray(frame)
             
        fps.update()  
        cv2.putText(frame,str(frame_num),(10,60),0,1.5,(0,255,255),1)
        frame_num+=1 
        out.write(frame)
        cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("result", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break
    else:
        break              
vid.release()  
out.release()
cv2.destroyAllWindows()
fps.stop()
print("Elapsed time: {:.2f}".format(fps.elapsed()))
print("FPS: {:.2f}".format(fps.fps()))

3 []
4 []
5 []
6 []
7 []
8 []
9 []
10 []
11 []
12 []
13 []
14 []
15 []
16 []
17 []
18 []
19 []
20 []
21 []
22 []
23 []
24 []
25 []
26 []
27 []
28 []
29 []
30 []
31 []
32 []
33 []
34 []
35 []
36 []
37 []
38 []
39 []
40 []
41 []
42 []
43 []
44 []
45 []
46 []
47 []
48 []
49 []
50 []
51 []
52 []
53 []
54 []
55 []
56 []
57 []
58 []
59 []
60 []
61 []
62 []
63 []
64 []
65 []
66 []
67 []
68 []
69 []
70 []
71 []
72 []
73 []
74 []
75 []
76 []
77 []
78 []
79 []
80 []
81 []
82 []
83 []
84 []
85 []
86 []
87 []
88 []
89 []
90 []
91 []
92 []
93 []
94 []
95 []
96 []
97 []
98 []
99 []
100 []
101 []
102 []
103 []
104 []
105 []
106 []
107 ['m:X-controller d:8.54']
108 ['m:X-controller d:6.00']
109 ['m:X-controller d:6.08']
110 ['m:X-controller d:6.00']
111 ['m:X-controller d:4.12']
112 ['m:X-controller d:3.61']
113 ['m:X-controller d:4.12']
114 ['m:X-controller d:3.16']
115 ['m:X-controller d:3.16']
116 []
117 ['m:X-controller d:3.16']
118 []
119 ['m:X-controller d:3.16']
120 ['m:X-controller d:4.12']
12

In [ ]:
store

In [ ]:
#if a[-1]>a[-2]
cr_lbl,pv_1_lbl = ['Workpiece', 'X-controller', 'xyz'], ['Spindle holder', 'Workpiece', 'X-controller', 'abc']
cr_xy,pv_1_xy = [(0,0),(0,0),(0,0)], [(0,0),(0,1),(1,1),(0,2),(0,3)]

cmn = [i for i in cr_lbl if i in pv_1_lbl]
cr_idx, pv_idx = [cr_lbl.index(i) for i in cmn], [pv_1_lbl.index(i) for i in cmn]
cr_lbl1, pv_1_lbl1 = [cr_lbl[i] for i in cr_idx], [pv_1_lbl[i] for i in pv_idx]
cr_xy1, pv_1_xy1 = [cr_xy[i] for i in cr_idx], [pv_1_xy[i] for i in pv_idx]

res = list(np.linalg.norm(np.array(cr_xy1)-np.array(pv_1_xy1),axis=1))

print(cr_xy1, pv_1_xy1)
print(pv_1_lbl1)

# print(cr_lbl1, pv_1_lbl1)
# res = list(np.sum(b_1, axis=0))

In [ ]:
a = 3.345654
print('val:{:.2f}'.format(a))

In [ ]:
a = ['X-controller', (40, 408)]
b = [['X-controller', (41, 410)], ['X-controller', (41, 410)]]
mx = max(len(a),len(b))
for i in range(2):
    if b[i]==a[i]

In [ ]:
wnd_lbl,wnd_xy = [], []
cr_lbl,cr_xy = ['a','b','c'], [(0,0),(0,0),(0,0)]
pv_1_lbl,pv_1_xy = ['x','a','y','b','c'], [(0,0),(0,1),(1,1),(0,2),(0,3)]


if len(cr_lbl)<len(pv_1_lbl):
    idx = [pv_1_lbl.index(i) for i in cr_lbl if i in pv_1_lbl]
    c_x = [pv_1_xy[i] for i in idx]
    res = list(np.linalg.norm(np.array(cr_xy)-np.array(c_x),axis=1))
    wnd_lbl.append(cr_lbl)
    wnd_xy.append(res)

#     for i in range(len(res)):
#         if res[i]>3:
#             cv2.putText(frame, "m:{} d:{:.2f}".format(cr_lbl[i], res[i]),(5, 35), 0, 1.5, (0,255,0),2)
#     bal = ["pv1 m:{} d:{:.2f}".format(cr_lbl[i], res[i]) for i in range(len(res)) if res[i]>3]
#     print(bal)
else:
    idx = [cr_lbl.index(i) for i in pv_1_lbl if i in cr_lbl]
    c_x = [cr_xy[i] for i in idx]
    res = list(np.linalg.norm(np.array(pv_1_xy)-np.array(c_x),axis=1))
    wnd_lbl.append([cr_lbl[i] for i in idx])
    wnd_xy.append(res)
#     for i in range(len(res)):
#         if res[i]>3:
#             cv2.putText(frame, "m:{} d:{:.2f}".format(cr_lbl[i], res[i]),(5, 35), 0, 1.5, (0,255,0),2)
#     bal = ["pv1 m:{} d:{:.2f}".format(cr_lbl[i], res[i]) for i in range(len(res)) if res[i]>3]
#     print(bal) 


print(wnd_lbl,wnd_xy)



In [ ]:
a = [(0,0),(0,0),(0,0)]
b = [(0,1),(0,2),(0,3)]

res = np.linalg.norm(np.array(a)-np.array(b), axis=1)
res


In [ ]:
x = np.array(a[1])
y = np.array(b[1])

res = np.linalg.norm(x-y)
res

In [ ]:
ex = [[0,0],[0,0],[0,0],[0,0],[0,0]]
all(len(m)==len(ex[0]) for m in ex)

In [ ]:
tst = [[1, 2, 3], [1, 2, 3], [1, 2, 3]]
tst = np.array(tst)
tst = list(np.sum(tst, axis=0))
tst

In [ ]:
a = [(1, 1),(168, 294)]
b = [(0, 0),(158, 236)]
mn = min(len(a),len(b))
m = np.array(a)
n = np.array(b)

res = np.linalg.norm(m-n, axis=1)
res

In [ ]:
lst = [[165, 302], [168, 294], [168, 288],[167, 282],[167, 276],[166, 272],[164, 269],[163, 265],[162, 262],[161, 259],
      [161, 256],[160, 253],[161, 251],[160, 248],[161, 244],[161, 240],[161, 238],[161, 236],[160, 235],[159, 236],
       [158, 236],[158, 237],[159, 236],[158, 236],[159, 236]]

for i in range(len(lst)-1):
    a = np.array(lst[i])
    b = np.array(lst[i+1])
    dis = np.linalg.norm(a-b)
    print(a,b,dis)



In [ ]:
os.chdir('D:/Desktop/OD/custom_data/images')
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
ffmpeg_extract_subclip('try_6_above.avi',215,220,'cut_1.mp4')
os.chdir(os.getcwd())

In [ ]:
distance =[0,5,7]
[playsound('audio.mp3') for i in distance if i>5]

In [ ]:
for i in lbl_idx:
    print(distance[i])

In [ ]:
a = np.array([(1,2), (3,6)])
b = np.array([(0,0), (0,0)])
# a-b
dis = np.linalg.norm(a-b, axis=1)
dis = list(dis)
print(type(dis))

In [ ]:
print('{} and {}'.format(2,3))

In [ ]:
a = np.array([(1,2)])
b = np.array([])
# a-b
dis = np.linalg.norm(a-b, axis=1)
dis

In [ ]:
numbers = [1, 2, 3, 7, 9]
window_size = 3

i = 0
moving_averages = []
while i < len(numbers) - window_size + 1:
    this_window = numbers[i : i + window_size]
get current window

    window_average = sum(this_window) / window_size
    moving_averages.append(window_average)
    i += 1

print(moving_averages)